# Pre-processing
1. Read miniseed data
2. Update header variables
3. Write SAC files

In [2]:
from obspy import UTCDateTime
import obspy
import os

# Define functions

In [15]:
def read_evt(evtf):
    """
    read event information
    """
    with open(evtf, 'r') as f:
        lines = f.readlines()[1:]
    events = []
    dnames = []
    for line in lines:
        line = line.strip()
        temp = line.split(',')
        t = UTCDateTime(temp[0])
        lat, lon = float(temp[1]), float(temp[2])
        depth = float(temp[3])
        dpu = temp[4]
        mag = float(temp[5])
        magt = temp[6]
        events.append([t, lat, lon, depth, dpu, mag, magt])
        dnames.append("%04d%02d%02d%02d%02d%02d%s" % (t.year, t.month, t.day, t.hour, t.minute, t.second, str(t.microsecond)[:3]))
    return dnames, events


def read_sta(staf):
    """
    read station information
    """
    with open(staf, 'r') as f:
        lines = f.readlines()
    stations, name = [], []
    for line in lines:
        line=line.strip()
        #if (not line) or line[0] == '#':
        #    continue
        temp = line.split(' ')
        name.append(temp[0])
        lat, lon = float(temp[1]), float(temp[2])
        stations.append([lat, lon])
    return name, stations


def mseed2sac(dnames, events, fnames, stations):
    for evt in zip(dnames, events):
        for sta in zip(fnames, stations):
            fpath = os.path.join(data_root, evt[0], sta[0] + '.mseed')
            try:
                st = obspy.read(fpath)
            except FileNotFoundError:
                print("Missing File: %s" % fpath)
                continue
            
            # convert to sac
            st.write('temp.sac', format='SAC')
            st = obspy.read('temp.sac')
            os.remove('./temp.sac')

            st[0].stats.sac.stla = sta[1][0]
            st[0].stats.sac.stlo = sta[1][1]
            st[0].stats.sac.evla = evt[1][1]
            st[0].stats.sac.evlo = evt[1][2]
            st[0].stats.sac.evdp = evt[1][3]
            st[0].stats.sac.mag = evt[1][5]
            st[0].stats.sac.o = evt[1][0] - st[0].stats.starttime
            outd = os.path.join(data_out, evt[0])
            if not os.path.exists(outd):
                os.mkdir(outd)
            st.write(os.path.join(outd, sta[0]+'.SAC'))

# Call functions

In [30]:
data_root = './miniseed/'
data_out = './sac'
evt_lst = './events.csv'
sta_lst = './station.dat'
if not os.path.exists(data_out):
    os.mkdir(data_out)
dnames, events = read_evt(evt_lst)
fnames, stations = read_sta(sta_lst)
mseed2sac(dnames, events, fnames, stations)

Missing File: ./miniseed/20190704173348530/CI.CCC..HHE.mseed
Missing File: ./miniseed/20190704173348530/CI.CCC..HHN.mseed
Missing File: ./miniseed/20190704173348530/CI.CCC..HHZ.mseed
Missing File: ./miniseed/20190704173348530/CI.CLC..HHE.mseed
Missing File: ./miniseed/20190704173348530/CI.CLC..HHN.mseed
Missing File: ./miniseed/20190704173348530/CI.CLC..HHZ.mseed
Missing File: ./miniseed/20190704173348530/CI.JRC2..HHE.mseed
Missing File: ./miniseed/20190704173348530/CI.JRC2..HHN.mseed
Missing File: ./miniseed/20190704173348530/CI.JRC2..HHZ.mseed
Missing File: ./miniseed/20190704173348530/CI.SRT..HHE.mseed
Missing File: ./miniseed/20190704173348530/CI.SRT..HHN.mseed
Missing File: ./miniseed/20190704173348530/CI.SRT..HHZ.mseed
Missing File: ./miniseed/20190704173348530/CI.TOW2..HHE.mseed
Missing File: ./miniseed/20190704173348530/CI.TOW2..HHN.mseed
Missing File: ./miniseed/20190704173348530/CI.TOW2..HHZ.mseed
Missing File: ./miniseed/20190704173348530/CI.WBM..HHE.mseed
Missing File: ./mi

# Benchmark
check the header variables

In [28]:
ref = './seismograms/'
flog = open('bench.log', 'w')
for d in os.listdir(ref):
    d2 = os.path.join(ref, d)
    for fname in os.listdir(d2):
        pref = os.path.join(d2, fname)
        psac = os.path.join(data_out, d, fname)
        tr0 = obspy.read(pref)[0]
        tr1 = obspy.read(psac)[0]
        s0 = tr0.stats
        s1 = tr1.stats
        flog.write("---------------------------------------------\n")
        flog.write("sourcetime stla stlo evla evlo evdp mag\n")
        flog.write(" ".join(map(lambda x: str(x), [s0.starttime+s0.sac.o-s0.sac.b, s0.sac.stla, s0.sac.stlo, s0.sac.evla, s0.sac.evlo, s0.sac.evdp, s0.sac.mag, '\n'])))
        flog.write(" ".join(map(lambda x: str(x), [s1.starttime+s1.sac.o-s1.sac.b, s1.sac.stla, s1.sac.stlo, s1.sac.evla, s1.sac.evlo, s1.sac.evdp, s1.sac.mag, '\n'])))
        flog.write("---------------------------------------------\n")